In [76]:
import importlib
import seaborn as sns

In [77]:
import numpy as np
import matplotlib.pyplot as plt
import helpers

In [78]:
importlib.reload(helpers)

<module 'helpers' from '/Users/daniildubinin/Documents/lecture-notes/MA1/ML/MLProject1/helpers.py'>

In [79]:
fields_to_drop = [
    "_STATE",
    "FMONTH",
    "IDATE",
    "IMONTH",
    "IDAY",
    "IYEAR",
    "DISPCODE",
    "SEQNO",
    "_PSU",
    "CTELENUM",
    "PVTRESD1",
    "STATERES",
    "CELLFON3",
    "LADULT",
    "NUMADULT",
    "NUMMEN",
    "NUMWOMEN",
    "CTELNUM1",
    "CELLFON2",
    "CADULT",
    "PVTRESD2",
    "CSTATE",
    "LANDLINE",
    "HHADULT",
    "HLTHPLN1",
    "MARITAL",
    "EDUCA",
    "RENTHOM1",
    "NUMHHOL2",
    "NUMPHON2",
    "CPDEMO1",
    "VETERAN3",
    "EMPLOY1",
    "CHILDREN",
    "INCOME2",
    "INTERNET",
    "MEDCOST",
    "USEEQUIP",
    "BLIND",
    "DECIDE", 
    "DIFFWALK",
    "DIFFDRES",
    "DIFFALON",
    "SEATBELT", 
    "IMFVPLAC", # ajouté
    "CAREGIV1",
    "CRGVREL1",
    "CRGVLNG1",
    "CRGVHRS1",
    "CRGVPRB1",
    "CRGVPERS",
    "CRGVHOUS",
    "CRGVEXPT",
    "VINOCRE2",
    "VIINSUR2",
    "CDHELP",
    "SXORIENT", # jusqu'ici
    "TRNSGNDR",
    "RCSGENDR", 
    "RCSRLTN2", #
    "QSTVER",
    "QSTLANG",
    "MSCODE",
    '_STSTR',
    '_STRWT',
    '_RAWRAKE',
    '_WT2RAKE',
    '_CHISPNC',
    '_DUALUSE',
    '_DUALCOR',
    '_LLCPWT',
    '_HCVU651',
    '_DRDXAR1',
    '_PRACE1',
    '_MRACE1',
    '_HISPANC',
    '_RACE',
    '_RACEG21',
    '_RACEGR3',
    '_RACE_G1',
    '_CHLDCNT',
    '_EDUCAG',
    '_INCOMG',
    '_RFSEAT2',
    '_RFSEAT3']


In [80]:
train_dict, test_dict, y_train, train_ids, test_ids = helpers.load_csv_data('data/dataset')

In [81]:
keep_dict = helpers.filter_features_by_nan(train_dict)

Keeping 200 features


In [82]:
train_clean, test_clean, final_keep = helpers.drop_and_keep_features(keep_dict, test_dict, fields_to_drop)

In [83]:
len(train_clean)

130

In [84]:
list(train_clean.keys())[0:41]

['GENHLTH',
 'PHYSHLTH',
 'MENTHLTH',
 'POORHLTH',
 'PERSDOC2',
 'CHECKUP1',
 'BPHIGH4',
 'BPMEDS',
 'BLOODCHO',
 'CHOLCHK',
 'TOLDHI2',
 'CVDSTRK3',
 'ASTHMA3',
 'CHCSCNCR',
 'CHCOCNCR',
 'CHCCOPD1',
 'HAVARTH3',
 'ADDEPEV2',
 'CHCKIDNY',
 'DIABETE3',
 'SEX',
 'WEIGHT2',
 'HEIGHT3',
 'QLACTLM2',
 'SMOKE100',
 'SMOKDAY2',
 'USENOW3',
 'ALCDAY5',
 'AVEDRNK2',
 'DRNK3GE5',
 'MAXDRNKS',
 'FRUITJU1',
 'FRUIT1',
 'FVBEANS',
 'FVGREEN',
 'FVORANG',
 'VEGETAB1',
 'EXERANY2',
 'EXRACT11',
 'EXEROFT1',
 'EXERHMM1']

# 1 - 5

- GENHLTH

# 1 - 30 (days) + 88 (answer is no)

- PHYSHLTH
- MENTHLTH
- POORHLTH

# 1 - 3
- PERSDOC2
- SMOKDAY2
- USENOW3

# 1 - 4, 8

- CHECKUP1

# 1 - 4
- BPHIGH4
- CHOLCHK
- DIABETE3

# 1 - 2
- BPMEDS
- BLOODCHO
- TOLDHI2
- CVDSTRK3
- ASTHMA3
- CHCSCNCR
- CHCOCNCR
- CHCCOPD1
- HAVARTH3
- ADDEPEV2
- SEX
- QLACTLM2
- SMOKE100
- EXERANY2

# 1 - 76

- AVEDRNK2 -> number of drinks 
- MAXDRNKS -> number of drinks

# 1-76 , 88

- DRNK3GE5 -> number of times

# A lot of discrete values

- FRUITJU1
- FRUIT1
- FVBEANS
- FVGREEN
- FVORANG
- VEGETAB1
- EXRACT11
- EXEROFT1
- EXERHMM1

# Continuous

- WEIGHT2 (can be in pounds and in kg at the same time)
- HEIGHT3 (can be ft and meters at the same time)
- ALCDAY5 (not continuous but 2 types: last 30 days or per day)

# First Selection/ Classification:

## (7,9) dont know or refused:

- GENHLTH: 7,9 don't know, refused
- PERSDOC2: 7,9 don't know, refused
- CHECKUP1: 7,9 don't know, refused
- BPHIGH4: 7,9 don't know, refused
- BPMEDS: 7,9 don't know, refused
- BLOODCHO: 7,9 don't know, refused
- CHOLCHK: 7,9 don't know, refused
- TOLDHI2: 7,9 don't know, refused
- CVDSTRK3: 7,9 don't know, refused
- ASTHMA3: 7,9 don't know, refused
- CHCSCNCR: 7,9 don't know, refused
- CHCOCNCR: 7,9 don't know, refused
- CHCCOPD1: 7,9 don't know, refused
- HAVARTH3: 7,9 don't know, refused
- ADDEPEV2: 7,9 don't know, refused
- CHCKIDNY: 7,9 don't know, refused
- DIABETE3: 7,9 don't know, refused
- QLACTLM2: 7,9 don't know, refused
- SMOKE100: 7,9 don't know, refused
- SMOKDAY2: 7,9 don't know, refused
- USENOW3: 7,9 don't know, refused
- EXERANY2: 7, 9

7_9 = ['GENHLTH','PERSDOC2', 'CHECKUP1','BPHIGH4','BPMEDS','BLOODCHO','CHOLCHK','TOLDHI2','CVDSTRK3','ASTHMA3','CHCSCNCR','CHCOCNCR','CHCCOPD1','HAVARTH3','ADDEPEV2','CHCKIDNY','DIABETE3','QLACTLM2','SMOKE100','SMOKDAY2','USENOW3','EXERANY2']

## (77, 99) don't know, refused:

- PHYSHLTH: 77, 99 don't know, refused
- MENTHLTH: 77, 99 don't know, refused
- POORHLTH: 77, 99 don't know, refused
- DRNK3GE5: 77, 99 dunno or refused
- MAXDRNKS: 77,99
- AVEDRNK2: 77,99 don't know, refused
- EXRACT11: 77, 99

77_99 = ['PHYSHLTH','MENTHLTH','POORHLTH','DRNK3GE5','MAXDRNKS','AVEDRNK2','EXRACT11']

## (7777, 9999) don't know, refused

- WEIGHT2: 7777 dunno, 9999 not sure
- HEIGHT3: 7777 dunno, 9999 not sure

7777_9999 = ['WEIGHT2','HEIGHT3']

## (777,999) don't know, refused

- ALCDAY5: 777, 999 don't know, refused
- FRUITJU1: 777, 999
- FRUIT1: 777, 999
- FVBEANS: 777, 999
- FVGREEN: 777, 999
- FVORANG: 777, 999
- VEGETAB1: 777, 999
- EXEROFT1: 777, 999
- EXERHMM1: 777, 999

777_999 = ['ALCDAY5','FRUITJU1','FRUIT1','FVBEANS','FVGREEN','FVORANG','VEGETAB1','EXEROFT1','EXERHMM1']





# What did:

- Reaplced don't know or refused answers by NaN's? OK
- Classify by continuous, discrete variables
- Handle realted variables? (some variables say if answer is X then go to variable Y)
- Same units (kg's and etc)
- Remove exract11 (pt être?)

In [127]:
invalid_dict = {
    (7,9): ['GENHLTH','PERSDOC2','CHECKUP1',
            'BPHIGH4','BPMEDS','BLOODCHO',
            'CHOLCHK','TOLDHI2','CVDSTRK3',
            'ASTHMA3','CHCSCNCR','CHCOCNCR',
            'CHCCOPD1','HAVARTH3','ADDEPEV2',
            'CHCKIDNY','DIABETE3','QLACTLM2',
            'SMOKE100','SMOKDAY2','USENOW3',
            'EXERANY2','CHOLCHK','DRNKANY5',
            '_RFBMI5','_CHOLCHK','_RFSMOK3',
            'PAMISS1_'],
    (77,99): ['PHYSHLTH','MENTHLTH','POORHLTH',
              'DRNK3GE5','MAXDRNKS','AVEDRNK2',
              'EXRACT11'],
    (777,999): ['ALCDAY5','FRUITJU1','FRUIT1',
                'FVBEANS','FVGREEN','FVORANG',
                'VEGETAB1','EXEROFT1','EXERHMM1','MAXVO2_',
                'FC60_'],
    (7777,9999): ['WEIGHT2','HEIGHT3'],
    (9): ['_VEGLT1','_TOTINDA','_PACAT1',
          '_PAINDX1','_PA150R2', '_PA300R2',
          '_PA30021', '_PASTRNG', '_PAREC1',
          '_PASTAE1', '_FLSHOT6', '_LMTACT1',
          '_LMTWRK1','_LMTSCL1','_PNEUMO2',
          '_AIDTST3', '_RFHLTH','_RFHYPE5',
          '_LTASTH1','_CASTHM1', '_ASTHMS1',
          '_SMOKER3','_RFBING5','_RFDRHV5',
          '_FRTLT1'],
    (99): ['PAFREQ1_', 'PAFREQ2_', 'STRFREQ_'],

    (99900): ['_DRNKWEK'],
    (14): ['_AGEG5YR'],
    (99999): ['WTKG3'],
    (900): ['DROCDY3','DROCDY3_'],
    (3): ['_AGE65YR']

    
}

In [128]:
helpers.replace_invalid_with_nan_inplace(train_clean, invalid_dict)

In [129]:
n_rows = len(next(iter(train_clean.values())))  # number of samples

for name, values in train_clean.items():
    if name in set(list(train_clean.keys())):
        nan_pct = np.isnan(values).sum() / n_rows * 100
        print(f'{name}: {nan_pct}')

GENHLTH: 0.27062032395203195
PHYSHLTH: 2.166181602084508
MENTHLTH: 1.6331692748411477
POORHLTH: 49.80175842260046
PERSDOC2: 0.406235238545111
CHECKUP1: 1.3549301354625383
BPHIGH4: 0.27976290246392493
BPMEDS: 59.89912688375212
BLOODCHO: 2.114983162417907
CHOLCHK: 14.660429396437442
TOLDHI2: 14.117360232831
CVDSTRK3: 0.224602678775504
ASTHMA3: 0.2998765751900894
CHCSCNCR: 0.2486781355234888
CHCOCNCR: 0.19930821155926676
CHCCOPD1: 0.4714522985966142
HAVARTH3: 0.5738491779298155
ADDEPEV2: 0.44189129474149363
CHCKIDNY: 0.29774330687064776
DIABETE3: 0.14597650357322445
SEX: 0.0
WEIGHT2: 7.00961494506834
HEIGHT3: 3.4622944824538684
QLACTLM2: 2.78787694089323
SMOKE100: 3.9465463909671326
SMOKDAY2: 58.39761073948223
USENOW3: 3.792036814116141
ALCDAY5: 4.868118304965944
AVEDRNK2: 52.88158837064013
DRNK3GE5: 52.84837033538026
MAXDRNKS: 53.748304813567586
FRUITJU1: 8.644612735611867
FRUIT1: 8.206073719658068
FVBEANS: 8.954241394547974
FVGREEN: 8.50046474774102
FVORANG: 8.825026284913221
VEGETAB1: 

In [130]:
len(train_clean)

130

* Transform extract en juste fait spor ou fait pas sport
*  \> 20% missing => drop

# Categories for variables:



# Keep variables only with <20% of missing data

- We bserve that a lot of feautures with \< 20% missing and other starting from \>30% -> we will drop them as a lot

# >20% variables

- POORHLTH
- BPMEDS
- SMOKDAY2
- AVEDRNK2
- DRNK3GE5
- MAXDRNKS
- EXRACT11
- EXEROFT1
- EXERHMM1
- EXRACT21
- EXEROFT2
- EXERHMM2
- LMTJOIN3
- ARTHDIS2
- ARTHSOCL
- JOINPAIN
- FLSHTMY2
- METVL11_
- METVL21_
- ACTIN11_
- ACTIN21_
- PADUR1_
- PADUR2_
- PAFREQ1_
- PAFREQ2_
- _MINAC11
- _MINAC21
- PAMIN11_
- PAMIN21_
- PA1MIN_
- PAVIG11_
- PAVIG21_
- PA1VIGM_
- _PNEUMO2

In [131]:
keep_dict = helpers.filter_features_by_nan(train_clean,threshold=20)

Keeping 95 features


In [132]:
n_rows = len(next(iter(keep_dict.values())))  # number of samples

for name, values in keep_dict.items():
    if name in set(list(keep_dict.keys())):
        nan_pct = np.isnan(values).sum() / n_rows * 100
        print(f'{name}: {nan_pct}')

GENHLTH: 0.27062032395203195
PHYSHLTH: 2.166181602084508
MENTHLTH: 1.6331692748411477
PERSDOC2: 0.406235238545111
CHECKUP1: 1.3549301354625383
BPHIGH4: 0.27976290246392493
BLOODCHO: 2.114983162417907
CHOLCHK: 14.660429396437442
TOLDHI2: 14.117360232831
CVDSTRK3: 0.224602678775504
ASTHMA3: 0.2998765751900894
CHCSCNCR: 0.2486781355234888
CHCOCNCR: 0.19930821155926676
CHCCOPD1: 0.4714522985966142
HAVARTH3: 0.5738491779298155
ADDEPEV2: 0.44189129474149363
CHCKIDNY: 0.29774330687064776
DIABETE3: 0.14597650357322445
SEX: 0.0
WEIGHT2: 7.00961494506834
HEIGHT3: 3.4622944824538684
QLACTLM2: 2.78787694089323
SMOKE100: 3.9465463909671326
USENOW3: 3.792036814116141
ALCDAY5: 4.868118304965944
FRUITJU1: 8.644612735611867
FRUIT1: 8.206073719658068
FVBEANS: 8.954241394547974
FVGREEN: 8.50046474774102
FVORANG: 8.825026284913221
VEGETAB1: 9.323296813811389
EXERANY2: 8.55349170311
STRENGTH: 8.730248221006597
FLUSHOT6: 9.32543008213083
PNEUVAC3: 9.454035686531457
HIVTST6: 9.776463955384216
_RFHLTH: 0.2706

In [133]:
len(keep_dict)

95

# Health

- **GENHLTH** - Would you say that in general your health is (excellend, good, poor) -> may be **biased**
- **PHYSHLTH** - physical illness and injury, for how many days during the past
30 days was your physical health not good
days during the past 30 days was your mental health not good **number of days => noisy variable**
- _RFHLTH - Adults with good or better health **more general and includes results of GENGLTH, and GENLTH too many valus and may be noisy**

# Check ups

- PERSDOC2 - Do you have one person you think of as your personal doctor or health care provider?
- CHECKUP1 - About how long has it been since you last visited a doctor for a routine checkup?

# Chronic Disease

- CHCCOPD1 - : (Ever told) you have Chronic Obstructive Pulmonary Disease or COPD, emphysema or chronic bronchitis?

# Other health realated

- HAVARTH3 - : (Ever told) you have some form of arthritis, rheumatoid arthritis, gout, lupus, or fibromyalgia? 

# Skin cancer/cancer

- **CHCSCNCR** - (Ever told) you had skin cancer?
- **CHCOCNCR** - (Ever told) you had any other types of cancer?

**unrelated for cardivascular (chatgpt)**

# Stroke 

- CVDSTRK3 - (Ever told) you had a stroke.

# Stress/depression

- ADDEPEV2 - (Ever told) you that you have a depressive disorder, including depression, major depression, dysthymia, or minor
depression?

- **MENTHLTH** - stress, depression, and problems with emotions, for how many **noisy and may be correlated with ADDEPEV2**

# Kidney

- CHCKIDNY - (Ever told) you have kidney disease? Do NOT include kidney stones, bladder infection or incontinence


# Diabet

- DIABETE3 - (Ever told) you have diabetes 

# Flu

- FLUSHOT6 - During the past 12 months, have you had either a flu shot or a flu vaccine that was sprayed in your nose?

# Pneumonia

- PNEUVAC3 - Have you ever had a pneumonia shot?

# Blood pressure

- _RFHYPE5 - Adults who have been told they have high blood pressure by a doctor, nurse, or other health professional
- **BPHIGH4** - Have you EVER been told by a doctor, nurse or other health professional that you have high blood pressure? **this one more noisy then first**

# Cholesterol

- _RFCHOL - Adults who have had their cholesterol checked and have been told by a doctor, nurse, or other health professional **same as BLOODCHO and TOLDHI2**
that it was high
- **_CHOLCHK** - Cholesterol check within past five years **may be correlated with check ups and cholesterol results**
- **BLOODCHO** - Have you EVER had your blood cholesterol checked?
- **CHOLCHK** - : About how long has it been since you last had your blood cholesterol checked? **noisy time dependent variable**
- **TOLDHI2** - Have you EVER been told by a doctor, nurse or other health professional that your blood cholesterol is high?


# ASTHMA

- _ASTHMS1 - Computed asthma status **this metric is more general and better as 3 below**
- **_CASTHM1** - Adults who have been told they currently have asthma
- **_LTASTH1** - Adults who have ever been told they have asthma
- **ASTHMA3** - (Ever told) you had asthma? ** same as _LTASTH1**

# HIV

BOTH LINKED

- **HIVTST6** - : Have you ever been tested for HIV?
- _AIDTST3 - Adults who have ever been tested for HIV

# Other

- SEX BINARY
- _AGEG5YR - catogories of ages by 14 level (18-24, 25-29, 30-34)  BINARY
- _AGE65YR - catogories of ages by 2 level (18-64, 65 or older) BINARY
- **_AGE80** - Imputed Age value collapsed above 80 (don't get what it is) BINARY
- **_AGE_G** - Six-level imputed age category (don't understand) BINARY
- MAXVO2_ - Estimated Age-Gender Specific Maximum Oxygen Consumption (seems important feauture) CONTINUOUS
- FC60_ - Estimated Functional Capacity CONTINUOUS
- _LMTSCL1 (can overlap with age, BMI, and comorbidities.) BINARY

# Metrics

- **WEIGHT2** - weight pounds (answer given by responder -> may be false)
- **HEIGHT3** - feet/inches height (answer given by responder -> may be false)
- **HTIN4** - height in inches
- **HTM4** - heigh in meters
- **WTKG3** - weight kilograms
- **_BMI5**
- _BMI5CA - body mass index by category (obese etc -> useful)
- **_RFBMI5** - Adults who have a body mass index greater than 25.00 (overweight or obese)

**use only body mass index by category more useful**

# Activities

- **QLACTLM2** - Are you limited in any way in any activities because of physical, mental, or emotional problems?  DON'T CARE
- **EXERANY2** - During the past month, other than your regular job, did you participate in any physical activities
- **STRENGTH** - During the past month, how many times per week or per month did you do physical activities or exercises to
STRENGTHEN your muscles?
- **_TOTINDA** - Adults who reported doing physical activity or exercise during the past 30 days other than their regular job
- **STRFREQ_** - Strength Activity Frequency per Week
- **PAMISS1_** - Missing Physical Activity Data
- _PACAT1 - Physical Activity Categories IMPORTANT
- _PAINDX1 - Physical Activity Index
- **_PA150R2** - DONT CARE
- **_PA300R2**: - DONT CARE
- **_PA30021** - DONT CARE
- _PASTRNG - Muscle Strengthening Recommendation
- _PAREC1 Aerobic and Strengthening Guideline
- _PASTAE1 - Aerobic and Strengthening (2-level)
- _LMTACT1  -Limited usual activities
- _LMTWRK1: - Limited work activities

# Smoking

- **SMOKE100** - Have you smoked at least 100 cigarettes in your entire life 
- **USENOW3** - Do you currently use chewing tobacco, snuff, or snus every day, some days, or not at all?
- _SMOKER3 - Four-level smoker status: Everyday smoker, Someday smoker, Former smoker, Non-smoke **more interesting variable as give at the same time former smokers and current smokers**
- **_RFSMOK3** - Adults who are current smokers

# Alcohol

- **ALCDAY5** - During the past 30 days, how many days per week or per month did you have at least one drink of any alcoholic 
- **DRNKANY5** - Adults who reported having had at least one drink of alcohol in the past 30 days
- **DROCDY3_** - Drink-occasions-per-day
- **_RFBING5** - Binge drinkers (males having five or more drinks on one occasion, females having four or more drinks on one
occasion)
- **_DRNKWEK** - Calculated total number of alcoholic beverages consumed per week
- _RFDRHV5 - Heavy drinkers (adult men having more than 14 drinks per week and adult women having more than 7 drinks per
week)

# Alimentation

- **FRUITJU1** - during past month, times per day, week or month drink 100 percent pure fruit juice. BINARY - REMOVE
- **FRUIT1** - during past month, times per day, week or month ate fruite BINARY - REMOVE
- **FVBEANS** - during past month, times per day, week or month ate beans BINARY
- **FVGREEN** - during past month, times per day, week or month ate green vegetables
- **FVORANG** - during past month, times per day, week or month ate orange colored vegetables BINARY
- **VEGETAB1** - during past month, times per day, week or month ate OTHER vegetables then above BINARY
- **FTJUDA1_** - Fruit juice in take in times per day BINARY
- **FRUTDA1_** - Fruit intake in times per day
- **BEANDAY_** - Bean intake in times per day
- **GRENDAY_** - Dark green vegetable intake in times per day
- **ORNGDAY_** - Orange-colored vegetable intake in times per day
- **VEGEDA1_** - OTHER vegetable intake in times per day
- **_MISFRTN** - The number of missing fruit responses
- **_MISVEGN** - The number of missing vegetable responses
- **_FRTRESP** - Missing any fruit responses
- **_VEGRESP** - Missing any vegetable responses
- _FRUTSUM - Total fruits consumed per day
- _VEGESUM - Total vegetables consumed per day
- **_FRTLT1** - Consume Fruit 1 or more times per day
- **_VEGLT1** - Consume Vegetables 1 or more times per day
- **_FRT16** - Reported consuming Fruit >16 per day
- **_VEG23** - Reported consuming Vegetables >23 per day
- **_FRUITEX** - Fruit Exclusion from analyses
- **_VEGETEX** - Vegetable Exclusion from analyses

**For fruits in order to have something interpretable and avoid multicolineairty we consider fruits and vegetables total per day FAIRE UN TRUC SCORE OU BINAIRE SI CONSOMME OU PAS (NOMBRE DE FRUITS C'EST TROP NOISY)**